In [ ]:
# pip install weaviate-client cohere

In [ ]:
import weaviate
import cohere

In [ ]:
WEAVIATE_API_KEY = "WfdvclBO4vWoiOfXCV1DjwOZUNZ08nr3NACb"
WEAVIATE_CLUSTER = "https://weaviate-hackathon-cluster-7l2iio8z.weaviate.network"
COHERE_API_KEY = "QnW41n57tHZl6r5EscOHPg1KbF6YrUAhpE7oJGSA"

In [ ]:
auth_config = weaviate.auth.AuthApiKey(api_key = WEAVIATE_API_KEY)
weaviate_url = WEAVIATE_CLUSTER

In [ ]:
client = weaviate.Client(url = weaviate_url,
additional_headers = {"X-Cohere-Api-Key":COHERE_API_KEY},
auth_client_secret = auth_config)

In [ ]:
client.is_ready()

True

In [ ]:
co = cohere.Client(COHERE_API_KEY)

In [22]:
# Create a conversation ID
import uuid
conversation_id = str(uuid.uuid4())

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break
    rag_response = ""
    results1 = client.query.get("Fiveyearhighlights", ["description", "year", "value"]).with_near_text({"concepts": [message]}).with_limit(5).do()

    results2 = client.query.get("Abbdata", ["doc_text"]).with_near_text({"concepts": [message]}).with_limit(3).do()

    results3 = client.query.get("bs_cf_spl_tables", ["bs_cf_spl_description", "bs_cf_spl_year", "bs_cf_spl_value"]).with_near_text({"concepts": [message]}).with_limit(10).do()

    greetings_prompt = "Hi, I am a FinAnalyticaAI chat bot for ABB and please ask me about prompt related to Financial Report."
    preamble_override = f"if prompt is greetings like hi, hello. then please respond by using following text:{greetings_prompt}. Use {results1}{results3} data to answer the prompt. if the data is not enough then use {results2} data and answer the following question in simple sentence " +message+f"also consider history of chat from following data {rag_response}"
    # Chatbot response
    response = co.chat(message=message,
                        preamble_override=preamble_override,
                        stream=True, #conversation_id=conversation_id,
                        conversation_id=conversation_id,
                        return_chat_history=True)


    print("Chatbot: ", end='')

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end='')
            rag_response = event.text
    print("\n")

Starting the chat. Type "quit" to end.

Chatbot: Hi! I'm a FinAnalyticaAI chat bot for ABB, and I'm here to help answer your questions about financial reports. I can provide you with data extracted from ABB's financial reports and other sources. How can I assist you today?



KeyboardInterrupt: Interrupted by user